In [2]:
import numpy as np
import glob
#import matplotlib.pyplot as plt
#%matplotlib inline
from skimage.io import imread
from skimage import img_as_float

-----

In [32]:
# (0-2, 4-6, 8-13, 15-20, 25-32, 38-43, 48-53, 60-)

In [23]:
def _read_fold(fold_idx, dataset_dir, debug=False):
    ages = dict() # debug
    dd = dict() # map column names to indices
    classes = {
        "(0, 2)": 0, "(4, 6)": 1, "(8, 12)": 2, "(15, 20)": 3, "(25, 32)": 4, "(38, 43)": 5, "(48, 53)": 6, "(60, 100)": 7
    }
    delim = "\t"
    # -----
    X, y = [], []
    with open("%s/../fold_%i_data.txt" % (dataset_dir, fold_idx)) as f:
        header = f.readline().rstrip().split(delim)
        for k in range(0, len(header)):
            dd[ header[k] ] = k
        for line in f:
            line = line.rstrip().split(delim)
            #print line
            #print line[dd["user_id"]], line[dd["original_image"]], line[dd["face_id"]], line[dd["age"]]
            img_loc = "%s/%s/landmark_aligned_face.%s.%s" % \
                (dataset_dir, line[dd["user_id"]], line[dd["face_id"]], line[dd["original_image"]])
            if line[dd["age"]] in classes:
                #print img_loc, classes[ line[dd["age"]] ]
                X.append(img_loc)
                y.append(classes[ line[dd["age"]] ])          
            if line[dd["age"]] not in ages:
                ages[ line[dd["age"]] ] = 0
            ages[ line[dd["age"]] ] += 1
    if debug:
        print ages
    return X, y

In [17]:
def load_xval_data(valid_fold_idx, dataset_dir):
    assert 0 <= valid_fold_idx <= 3
    tot_folds = ["fold_0_data.txt", "fold_1_data.txt", "fold_2_data.txt", "fold_3_data.txt"]
    training_folds_idxs = [ i for i in range(0, len(tot_folds)) if i != valid_fold_idx ]
    #valid_fold = tot_folds[valid_fold_idx]
    training_X = []
    training_y = []
    for training_fold_idx in training_folds_idxs:
        x, y = _read_fold(training_fold_idx, dataset_dir)
        training_X += x
        training_y += y
    valid_X, valid_y = _read_fold(valid_fold_idx, dataset_dir)
    training_X = np.asarray(training_X)
    valid_X = np.asarray(valid_X)
    training_y = np.asarray(training_y, dtype="int32")
    valid_y = np.asarray(valid_y, dtype="int32")
    return training_X, training_y, valid_X, valid_y

In [20]:
def load_all_data(dataset_dir):
    training_folds_idxs = [ i for i in range(0, 4) ]
    #valid_fold = tot_folds[valid_fold_idx]
    training_X = []
    training_y = []
    for training_fold_idx in training_folds_idxs:
        x, y = _read_fold(training_fold_idx, dataset_dir)
        training_X += x
        training_y += y
    training_X = np.asarray(training_X)
    training_y = np.asarray(training_y, dtype="int32")
    return training_X, training_y

In [29]:
def load_pre_split_data(dataset_dir):
    all_X, all_y = load_all_data(dataset_dir)
    rnd = np.random.RandomState(0)
    idxs = [x for x in range(len(all_y))]
    rnd.shuffle(idxs)
    train_idxs = idxs[0:int(0.9*len(idxs))]
    valid_idxs = idxs[int(0.9*len(idxs))::]
    return all_X[train_idxs], all_y[train_idxs], all_X[valid_idxs], all_y[valid_idxs]

In [37]:
#xt, yt, xv, yv = load_pre_split_data("/Volumes/CB_RESEARCH/adience_face/aligned/")

In [38]:
#len(xt), len(yt), len(xv), len(yv)

In [39]:
#from collections import Counter
#Counter(yv.tolist()).most_common()

----